<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/CSV_to_UVI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hier wird mit Hilfe der vorverarebiteten Messdaten der UVI berechnet:

**Dokumentenname: CSV_to_UVI.ipynb**

Es werden die vorverarbeiteten UV-Spektren in einen UVI und in die UVA und UVB Strahlung aggregiert. Zu Beginn muss das Skript *UV_Measurement_to_CSV* ausgeführt werden, um die Spektren in einen Dataframe umzuwandeln. Aufgrund der großen Datenmenge geschieht die Verarbeitung für jeden Monat getrennt.

## Import der benötigten Module:

In [1]:
# Import der benötigten Module
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Mount der Google-Drive:

In [2]:
# Mount google drive
from google.colab import drive
from google.colab import files

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Monat der Exportiert werden sollen:**

22.06
22.07
22.08
22.09
22.10
22.11
22.12
23.01
23.02
23.03
23.04
23.05
23.06
23.07
23.08

In [3]:
month = '22.06'

## Daten einlesen:

In [4]:
# Import der CSV mit den rohen Messdaten
# Pfad zur CSV-Datei auf Google Drive
drive_path = '/content/drive/My Drive/Colab_Notebooks/CSV_Messdaten/' + month

# CSV-Datei einlesen und DataFrame erstellen
df_Messdaten = pd.read_csv(drive_path)

## Definetion der UV-Gewichtungsfunktion und UVA und UVB Bereich:

In [5]:
# Gewichtungsfunktion für den UVI
def weight_function(Wellenlaenge):
    if Wellenlaenge <= 298:
        return 1
    elif Wellenlaenge > 298 and Wellenlaenge <= 328:
        return 10**(0.094 * (298.0 - Wellenlaenge))
    elif Wellenlaenge > 328 and Wellenlaenge <= 400:
        return 10**(0.015 * (139.0 - Wellenlaenge))
    else:
        return 0

In [6]:
# Bereich von Spalte x bis Spalte y (einschließlich y)
x_UVI = '290.0'
y_UVI = '400.0'
selected_columns_UVI = df_Messdaten.columns[(df_Messdaten.columns.get_loc(x_UVI)):(df_Messdaten.columns.get_loc(y_UVI)+1)]

In [7]:
# UVB Bereich von 290 bis 320.1 nm
x_UVB = '290.0'
y_UVB = '315.0'
selected_columns_UVA = df_Messdaten.columns[(df_Messdaten.columns.get_loc(x_UVB)):(df_Messdaten.columns.get_loc(y_UVB)+1)]

In [8]:
# UVA Bereich von 320.1 bis 400 nm
x_UVA = '315.1'
y_UVA = '400.0'
selected_columns_UVB = df_Messdaten.columns[(df_Messdaten.columns.get_loc(x_UVA)):(df_Messdaten.columns.get_loc(y_UVA)+1)]

## Erythemgewichtete UV-Messwerte, UVI und UVA UVB bestimmen:

In [9]:
# Erythemgewichtete UV-Messwerte
df_Gewichtet = []
df_Gewichtet = df_Messdaten[selected_columns_UVI].apply(lambda col: col * weight_function(float(col.name)), axis=0)

In [10]:
def integrate_row(row):
    x_values = np.array([float(col) for col in row.index])  # X-Werte aus Spaltennamen extrahieren
    y_values = np.array(row)
    integral = np.trapz(y_values, x_values)
    return integral

In [11]:
def calculate_UV_ABC(df):
    return df.sum(axis=1)

In [12]:
# Summe der Messwerte über das UVA und UVB Spektrum
UVA = calculate_UV_ABC(df_Messdaten[selected_columns_UVA])
UVB = calculate_UV_ABC(df_Messdaten[selected_columns_UVB])
# Spalte umbenennen
UVA.name = 'UVB'
UVB.name = 'UVB'

In [13]:
ser_UVI = []
df_UVI = []

# Integration der gewichteten UV-Messwerte
ser_UVI = df_Gewichtet.apply(integrate_row, axis=1)
# Umwandeln der Serie in einen Dataframe
df_UVI = pd.DataFrame(ser_UVI, columns=['erythem'])

In [15]:
# Dem Dataframe die Werte für UVA und UVB hinzufügen
df_UV = pd.concat([df_UVI, UVA, UVB], axis=1)

In [16]:
type(df_Gewichtet), type(df_UV)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

## Abspeichern der UVI, UVA, UVB Messwerte in einen Dataframe:

In [17]:
# Exportieren
columns_to_copy = ['Datetime', 'Datum', 'Uhrzeit', 'Messzeitpunkt', 'UVA', 'UVB']

for col in columns_to_copy:
  df_Gewichtet.insert(0, col, df_Messdaten[col])
  df_UV.insert(0, col, df_Messdaten[col])
  df_UV['UVI'] = df_UV['erythem'].copy() * 40,

ValueError: Length of values (1) does not match length of index (7308)

In [ ]:
df_UV

In [ ]:
def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstelle den vollen Pfad zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichere den DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=False)

    print(f'Der DataFrame wurde als {filename} in {folder_path} auf Google Drive gespeichert.')

In [ ]:
plt.plot(df_UV['UVI'])

In [ ]:
# save_dataframe_to_drive(df_UV, '/content/drive/My Drive/Colab_Notebooks/CSV_UVI', month)

In [ ]:
# save_dataframe_to_drive(df_Gewichtet, '/content/drive/My Drive/Colab_Notebooks/CSV_Gewichtet', month)